<a href="https://colab.research.google.com/github/wassimchouchen/Automatic-Speech-Recognition-/blob/main/Transcription%26Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null
!pip install pydub
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding( 
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib
import pydub

from sklearn.cluster import AgglomerativeClustering
import numpy as np


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
num_speakers = 2 
language = 'any' 
model_size = 'large'

In [3]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving conv.mp3 to conv (2).mp3


In [4]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'
# Load the audio file
audio_file = pydub.AudioSegment.from_wav(path)
audio_file = audio_file.set_channels(1)
audio_file.export(path, format="wav")
with wave.open(path) as f:
  print(f.getnchannels())

1


In [5]:
model = whisper.load_model(model_size)

In [6]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
segments

In [8]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [9]:
audio = Audio()
import pyannote.audio.utils

# Convert the audio recording to mono
def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [10]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [11]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
segments

In [13]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="utf-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:])
f.close()

In [14]:
print(open("transcript.txt", "r", encoding="utf-8").read())


SPEAKER 2 0:00:00
podcastfrancaisfacile.com
SPEAKER 1 0:00:04
Bonjour. Bonjour. Installez-vous je vous en prie.Merci. Alors qu'est-ce qui vous amène ?
SPEAKER 2 0:00:14
Eh bien j'ai des douleurs au ventre, au niveau de l'estomac.
SPEAKER 1 0:00:19
Vous avez mal à l'estomac ?
SPEAKER 2 0:00:22
En fait je ne suis pas sûr que ce soit à l'estomac.
SPEAKER 1 0:00:26
Vous avez mal en permanence ?
SPEAKER 2 0:00:29
Non, ce n'est pas tout le temps, c'est surtout après manger.
SPEAKER 1 0:00:33
Comment est la douleur ?
SPEAKER 2 0:00:36
Ce n'était pas très fort au début, mais depuis quelques jours c'est très aigu.
SPEAKER 1 0:00:43
Quand ont commencé les premières douleurs ?
SPEAKER 2 0:00:47
C'était il y a deux mois environ.Je pensais que ça passerait, mais finalement c'est de pire en pire.
SPEAKER 1 0:00:57
Bien, on va regarder ça.Enlevez votre chemise et allongez-vous.
